# GloFAS - Attributes from meteorology
***

**_Autor:_** Chus Casado Rodríguez<br>
**_Fecha:_** 14-10-2024<br>

**Introduction:**<br>
This notebook creates the static attributes for the catchmens of the reservoirs in the ResOpsUS dataset.

**To do:**<br>

In [2]:
import os
os.environ['USE_PYGEOS'] = '0'
from pathlib import Path
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
from tqdm.notebook import tqdm
from datetime import datetime
from shapely.geometry import Point
import yaml

from lisfloodreservoirs.utils.plots import plot_attributes
from lisfloodreservoirs.utils import DatasetConfig
from lisfloodreservoirs.utils.utils import duration_precip_indices

import sys
sys.path.append('../../../lisflood-utilities/src')
from lisfloodutilities.catchstats import catchment_statistics

## Configuration

In [ ]:
cfg = DatasetConfig('config_dataset.yml')

PATH_METEO = cfg.PATH_RESOPS / 'ancillary' / 'catchstats' / 'meteo'

PATH_PLOTS = cfg.PATH_ATTRS / 'plots'
PATH_PLOTS.mkdir(parents=False, exist_ok=True)

## Data

### Reservoirs

In [ ]:
# load TXT used as input for `cutmaps`
reservoirs = pd.read_csv(cfg.PATH_RESOPS / 'ancillary' / 'cutmaps' / 'points.txt', sep='\t', header=None)#.squeeze()
reservoirs.columns = ['lon', 'lat', 'GRAND_ID']
reservoirs.set_index('GRAND_ID', drop=True, inplace=True)

### Meteorology

In [ ]:
# load areal meteo time series
meteo = {}
variables = [item.stem for item in PATH_METEO.glob('*') if item.is_dir()]
for var in variables:
    da = xr.open_mfdataset(f'{PATH_METEO}/{var}/*.nc')[f'{var}_mean'].compute()
    da.name = var
    da = da.rename({'id': 'ID'})
    meteo[var] = da
meteo = xr.Dataset(meteo)
meteo = meteo.rename({'tp': 'precip', '2t': 'temp', 'e0': 'pet'})

# compute precipitation as snowfall
meteo['snow'] = meteo['precip'].where(meteo['temp'] > 1, 0)

## Attributes

In [ ]:
# average of all meteo variables
attrs = meteo.mean('time').to_pandas()
attrs.columns = [f'{col}_mean' for col in attrs.columns]
attrs.index.name = 'GRAND_ID'

# indices
attrs['aridity'] = attrs.precip_mean / attrs.pet_mean
attrs['frac_snow'] = attrs.snow_mean / attrs.precip_mean
attrs['moisture_index'] = attrs.precip_mean - attrs.pet_mean
precip_monthly = meteo['precip'].resample({'time': '1M'}).sum().groupby('time.month').mean()
precip_annual = meteo['precip'].resample({'time': 'A'}).sum().mean('time')
attrs['seasonality'] = ((precip_monthly.max('month') - precip_monthly.min('month')) / precip_annual).to_pandas()

# high and dry precipitation indices
precip_extremes = {
    'high': 20, # mm
    'low': 1 # mm
}
for key, value in precip_extremes.items():
    if key == 'high':
        mask_precip = meteo['precip'] > value
    elif key == 'low':
        mask_precip = meteo['precip'] < value
    attrs[f'{key}_precip_freq'] = mask_precip.sum('time') / len(meteo.time)
    attrs[f'{key}_precip_dur'] = duration_precip_extremes(mask_precip.to_pandas())

In [ ]:
# plot attributes
r = 5
extent = [np.floor(reservoirs.lon.min() / r) * r, np.ceil(reservoirs.lon.max() / r) * r,
          np.floor(reservoirs.lat.min() / r) * r, np.ceil(reservoirs.lat.max() / r) * r]
plot_attributes(
    attrs,
    reservoirs.lon,
    reservoirs.lat,
    ncols=4,
    extent=extent,
    save=PATH_PLOTS / f'maps_climate.jpg'
)

In [ ]:
# export
attrs.to_csv(cfg.PATH_ATTRS / 'climate_indices.csv', float_format='%.6f')